In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import sys
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as datetime

In [2]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from torch.optim.lr_scheduler import StepLR

from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
ds = xr.open_dataset('/home/sachin/Documents/NIPR/Research/Data/AMPERE/processed/ampere_omni_2010.nc')
ds

<xarray.Dataset> Size: 2GB
Dimensions:         (dt: 220129, mlat: 50, mlt: 24)
Coordinates:
  * dt              (dt) datetime64[ns] 2MB 2010-03-01 ... 2010-12-31T23:58:00
  * mlat            (mlat) float64 400B 50.0 50.82 51.63 ... 88.37 89.18 90.0
  * mlt             (mlt) float64 192B 0.0 1.043 2.087 3.13 ... 21.91 22.96 24.0
Data variables:
    jPar            (dt, mlat, mlt) float64 2GB ...
    BX_GSE          (dt) float64 2MB ...
    BY_GSE          (dt) float64 2MB ...
    BZ_GSE          (dt) float64 2MB ...
    flow_speed      (dt) float64 2MB ...
    proton_density  (dt) float64 2MB ...
    AL_INDEX        (dt) int32 881kB ...
    AU_INDEX        (dt) int32 881kB ...
    SYM_H           (dt) int32 881kB ...
    ASY_H           (dt) int32 881kB ...
    F10.7           (dt) float64 2MB ...
    Kp              (dt) int32 881kB ...

In [6]:
# Initialize scalers for the target variable and input variables
target_scaler = MinMaxScaler()
input_scaler = MinMaxScaler()

# Extract the target variable and reshape for scaling
target_var = ds['jPar'].values  # shape (22320, 50, 24)
target_var_reshaped = target_var.reshape(-1, 1)
target_var_scaled = target_scaler.fit_transform(target_var_reshaped).reshape(-1, 50, 24)

# Extract and scale input variables (variables that are dependent only on 'dt')
input_vars = ['BX_GSE', 'BY_GSE', 'BZ_GSE', 'flow_speed', 'proton_density', 'AL_INDEX', 'AU_INDEX', 'SYM_H', 'ASY_H', 'F10.7', 'Kp']
input_data = np.array([ds[var].values for var in input_vars]).T  # shape (22320, number_of_vars)
input_data_scaled = input_scaler.fit_transform(input_data)

def create_sequences(target_data, input_data, lookback=30):
    X, y = [], []
    for i in range(len(target_data) - lookback):
        X.append(input_data[i:i+lookback].T)
        y.append(target_data[i+lookback])

    return np.array(X), np.array(y)

lookback = 30
X, y = create_sequences(target_var_scaled, input_data_scaled, lookback=lookback)

X.shape, y.shape

((220099, 11, 30), (220099, 50, 24))

In [7]:
train_ratio = 0.8
val_ratio = 0.1
test_ratio = 0.1

# Calculate split indices
train_idx = int(len(X) * train_ratio)
val_idx = int(len(X) * (train_ratio + val_ratio))

# Perform the split
X_train, X_val, X_test = X[:train_idx], X[train_idx:val_idx], X[val_idx:]
y_train, y_val, y_test = y[:train_idx], y[train_idx:val_idx], y[val_idx:]

In [8]:

# Convert data to PyTorch tensors and move to device
X_train = torch.tensor(X_train, dtype=torch.float32).to(device)
y_train = torch.tensor(y_train, dtype=torch.float32).to(device)
X_val = torch.tensor(X_val, dtype=torch.float32).to(device)
y_val = torch.tensor(y_val, dtype=torch.float32).to(device)
X_test = torch.tensor(X_test, dtype=torch.float32).to(device)
y_test = torch.tensor(y_test, dtype=torch.float32).to(device)

# Create DataLoader
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=False)

val_dataset = TensorDataset(X_val, y_val)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [9]:
# Define the LSTM model
class LSTMModel(nn.Module):
    def __init__(self):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size=30, hidden_size=50, num_layers=2, batch_first=True)
        self.fc = nn.Linear(50, 24 * 50)
        
    def forward(self, x):
        x, _ = self.lstm(x)
        x = self.fc(x[:, -1, :])
        x = x.view(-1, 50, 24)
        return x

# Initialize the model, loss function and optimizer
model = LSTMModel().to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [10]:
# Train the model
num_epochs = 100

for epoch in range(num_epochs):
    model.train()
    for X_batch, y_batch in train_loader:
        # Forward pass
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    # Validation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            val_loss += loss.item()
    
    val_loss /= len(val_loader)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.3e}, Validation Loss: {val_loss:.3e}')


Epoch [1/100], Loss: 1.655e-04, Validation Loss: 1.625e-04
Epoch [2/100], Loss: 1.591e-04, Validation Loss: 1.470e-04
Epoch [3/100], Loss: 1.517e-04, Validation Loss: 1.391e-04
Epoch [4/100], Loss: 1.399e-04, Validation Loss: 1.347e-04
Epoch [5/100], Loss: 1.365e-04, Validation Loss: 1.319e-04
Epoch [6/100], Loss: 1.374e-04, Validation Loss: 1.299e-04
Epoch [7/100], Loss: 1.396e-04, Validation Loss: 1.308e-04
Epoch [8/100], Loss: 1.403e-04, Validation Loss: 1.292e-04
Epoch [9/100], Loss: 1.430e-04, Validation Loss: 1.303e-04
Epoch [10/100], Loss: 1.413e-04, Validation Loss: 1.293e-04
Epoch [11/100], Loss: 1.400e-04, Validation Loss: 1.287e-04
Epoch [12/100], Loss: 1.392e-04, Validation Loss: 1.286e-04
Epoch [13/100], Loss: 1.385e-04, Validation Loss: 1.285e-04
Epoch [14/100], Loss: 1.368e-04, Validation Loss: 1.283e-04
Epoch [15/100], Loss: 1.344e-04, Validation Loss: 1.272e-04
Epoch [16/100], Loss: 1.331e-04, Validation Loss: 1.259e-04
Epoch [17/100], Loss: 1.320e-04, Validation Loss:

In [ ]:
# Evaluate the model on the test set
model.eval()
model = model.cpu()  # Move model to CPU
X_test = X_test.cpu()  # Move test data to CPU
y_test = y_test.cpu()  # Move test targets to CPU

with torch.no_grad():
    predictions = model(X_test)
    test_loss = criterion(predictions, y_test)
    print(f'Test Loss: {test_loss.item():.4f}')

# Rescale predictions back to original scale
predictions_rescaled = target_scaler.inverse_transform(predictions.numpy().reshape(-1, 1)).reshape(-1, 50, 24)
y_test_rescaled = target_scaler.inverse_transform(y_test.numpy().reshape(-1, 1)).reshape(-1, 50, 24)

sample_lat_idx = 10  # example index
sample_lon_idx = 10 # example index

plt.figure(figsize=(10, 6))
plt.plot(y_test_rescaled[:, sample_lat_idx, sample_lon_idx], label='Actual')
plt.plot(predictions_rescaled[:, sample_lat_idx, sample_lon_idx], label='Predicted')
plt.legend()
plt.show()


In [ ]:
RMSE = np.sqrt(np.mean((predictions_rescaled - y_test_rescaled)**2))
NRMSE = RMSE / (np.max(y_test_rescaled) - np.min(y_test_rescaled))
R = np.corrcoef(predictions_rescaled.flatten(), y_test_rescaled.flatten())[0, 1]

print(f'RMSE: {RMSE:.4f}')
print(f'NRMSE: {NRMSE:.4f}')
print(f'R: {R:.4f}')